In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

## Sleep Accuracy

In [2]:
import pandas as pd

results_list = []
XTIMER_BACKOFFS=[40, 30, 22]
for i, backoff in enumerate(XTIMER_BACKOFFS, start=1):
    file = f"data/sleep_accuracy/xunit_xtimer_usleep_set_XTIMER_BACKOFF_{backoff}_now.xml"
    root = ET.parse(file).getroot()

    for d in root.find("testcase[@name='Measure Xtimer Set Accuracy Microseconds Template']").findall(".//property"):
        duration = d.get("name").split('-')[-1]
        results = [float(v["diff"]) - (int(duration) / 1000_000) for v in eval(d.get("value"))]    # diff from sleep time
        results_list.append({
            'type': 'xtimer_set',
            'backoff': backoff,
            'duration': duration,
            'results': results,
            'mean': np.mean(results),
            'max': np.amax(results),
            'min': np.amin(results)
        })
    

    for d in root.find("testcase[@name='Measure Xtimer Usleep Accuracy Microseconds Template']").findall(".//property"):
        duration = d.get("name").split('-')[-1]
        results = [float(v["diff"]) - (int(duration) / 1000_000) for v in eval(d.get("value"))]    # diff from sleep time
        results_list.append({
            'type': 'xtimer_usleep',
            'backoff': backoff,
            'duration': duration,
            'results': results,
            'mean': np.mean(results),
            'max': np.amax(results),
            'min': np.amin(results)
        })

sleep_accuracy = pd.DataFrame.from_dict(results_list)


In [196]:
xtimer_set_traces = []
xtimer_usleep_traces = []
for backoff in sleep_accuracy['backoff'].unique():
    xtimer_set = sleep_accuracy.loc[sleep_accuracy['type'] == 'xtimer_set']
    data = xtimer_set[xtimer_set['backoff'] == backoff]
    xtimer_set_traces.append(go.Scatter(x=data['duration'], y=data['mean'], name=f'xtimer_set / {backoff}'))

    xtimer_set = sleep_accuracy.loc[sleep_accuracy['type'] == 'xtimer_usleep']
    data = xtimer_set[xtimer_set['backoff'] == backoff]
    xtimer_usleep_traces.append(go.Scatter(x=data['duration'], y=data['mean'], name=f'xtimer_usleep / {backoff}'))


sleep_accuracy_fig = go.Figure()
sleep_accuracy_fig.add_traces(xtimer_set_traces)
sleep_accuracy_fig.add_traces(xtimer_usleep_traces)
sleep_accuracy_fig.update_layout(
    title="Sleep Accuracy with multiple XTIMER_BACKOFF",
    xaxis_title="Specified Sleep Duration [us]",
    yaxis_title="Real Sleep Duration [s]",
)

# # add range slider
# sleep_accuracy_fig.update_layout(
#     xaxis=dict(
#         rangeslider=dict(
#             visible=True,
#         )
#     )
# )

# sleep_accuracy_fig.update_layout(
#     legend=dict(orientation="h",yanchor="bottom",y=0.99)
# )
sleep_accuracy_fig.write_html("docs/sleep_accuracy.html")
go.FigureWidget(sleep_accuracy_fig)

FigureWidget({
    'data': [{'name': 'xtimer_set / 40',
              'type': 'scatter',
              'uid': …

## Sleep Jitter

In [169]:
suffix = "10-durch1"
file = f"data/sleep_jitter/xunit.xml"
root = ET.parse(file).getroot()

sleep_jitter = pd.DataFrame()
for testcase in root.findall("testcase[@classname='tests_gpio_overhead.Sleep Jitter']"):
    duration = DURATION
    traces = {}
    for d in testcase.findall('.//property'):
        name = d.get('name').split('-')
        # if name[-1] == 'focus':
        #     intervals = int(int(eval(d.get('value'))[0]) / 1000000)
        if 'intervals' in name:
                intervals = list(map(lambda i: int(i), eval(d.get('value'))))
        elif 'trace' in name:
            repeat_count = int(name[-1])
            traces[repeat_count] = eval(d.get('value'))

    for n in traces.keys():
        sleep_jitter = sleep_jitter.append(
            pd.DataFrame.from_dict({
                'repeat_count': n,
                # 'time': [i for i in range(1, len(traces[n]) + 1)],
                'trace_milli': map(lambda x: x * 1000, traces[n]),
                'background_timers': [len(intervals)] * len(traces[n]),
        }))
# sleep_jitter

In [168]:
# sleep jitter stats min/max/avg/std

sleep_jitter_stats_fig = px.violin(sleep_jitter, x='background_timers', y='trace_milli',
    points='all',
    color='background_timers',
    orientation='v',
    title=f"Sleep Jitter Statistics {suffix}",
    labels={
        "trace": "Sleep Duration (Seconds)",
        "trace_milli": "Sleep Duration (Milliseconds)",
        "background_timers": "No. Background Timers",
        "count": "Sample Percentage",
    },
)
# sleep_jitter_stats_fig.update_layout(
#     legend=dict(orientation="h", yanchor="bottom", y=0.99),
# )

# sleep_jitter_stats_fig.write_html(f'docs/sleep_jitter_stats_{suffix}.html')
go.FigureWidget(sleep_jitter_stats_fig)
    

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'box': {'visible': False},
              …

In [194]:
# sleep jitter stats min/max/avg/std

sleep_jitter_stats_fig = px.histogram(sleep_jitter, x='trace_milli',
    marginal='rug',
    histnorm='percent',
    color='background_timers',
    title=f"Sleep Jitter Statistics {suffix}",
    labels={
        "trace": "Sleep Duration (Seconds)",
        "trace_milli": "Sleep Duration (Milliseconds)",
        "background_timers": "No. Background Timers",
        "count": "Sample Percentage",
    },
)
# sleep_jitter_stats_fig.update_layout(
#     legend=dict(orientation="h", yanchor="bottom", y=0.99),
# )

# sleep_jitter_stats_fig.write_html(f'docs/sleep_jitter_stats_{suffix}.html')
go.FigureWidget(sleep_jitter_stats_fig)
    

FigureWidget({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'histnorm'…

In [191]:
sleep_jitters = {}
for divisor in [1, 10, 100, 1000]:
    file = f"data/sleep_jitter/old/xunit-10-durch{divisor}.xml"
    root = ET.parse(file).getroot()

    df = pd.DataFrame()
    for testcase in root.findall("testcase[@classname='tests_gpio_overhead.Sleep Jitter']"):
        duration = DURATION
        traces = {}
        for d in testcase.findall('.//property'):
            name = d.get('name').split('-')
            # if name[-1] == 'focus':
            #     intervals = int(int(eval(d.get('value'))[0]) / 1000000)
            if 'intervals' in name:
                intervals = list(map(lambda i: int(i), eval(d.get('value'))))
            elif 'trace' in name:
                repeat_count = int(name[-1])
                traces[repeat_count] = eval(d.get('value'))[2:]

        for n in traces.keys():
            df = df.append(pd.DataFrame.from_dict({
                    'repeat_count': n,
                    'trace': traces[n],
                    'trace_milli': map(lambda x : x * 1000, traces[n]),
                    'background_timers': [len(intervals)] * len(traces[n]),
            }))
        sleep_jitters[divisor] = df

In [180]:
from plotly.subplots import make_subplots

sleep_jitter_combined_figs = go.Figure()

for divisor in sleep_jitters.keys():
    sleep_jitter_combined_figs.add_trace(go.Violin(
        x=sleep_jitters[divisor]['background_timers'],
        y=sleep_jitters[divisor]['trace_milli'],
        name=divisor,
    ))

sleep_jitter_combined_figs.update_layout(
    title_text="Sleep Jitters Combined",
    legend_title_text='Inteval Divisor',
    xaxis=dict(
        title="No. Background Timers",
    ),
    yaxis=dict(
        title="Sleep Duration (Milliseconds)"
    )
    
)
# sleep_jitter_combined_figs.write_html('docs/sleep_jitter_stats_combined.html')
go.FigureWidget(sleep_jitter_combined_figs)

FigureWidget({
    'data': [{'name': '1',
              'type': 'violin',
              'uid': '058fae92-7bce-…